<a href="https://colab.research.google.com/github/vinayakankasi/Beginner-Python-Projects/blob/main/EduTutor_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install gardio toruch transformers

ERROR: Could not find a version that satisfies the requirement gardio (from versions: none)
ERROR: No matching distribution found for gardio


In [5]:
!pip install gradio torch transformers

In [7]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "ibm-granite/granite-3.2-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
    device_map="auto" if torch.cuda.is_available() else None
    )
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

def generate_response(prompt, max_length=512):
  inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
  if torch.cuda.is_available():
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
      outputs = model.generate(
          **inputs,
          max_length=max_length,
          temperature=0.7,
          do_sample=True,
          pad_token_id=tokenizer.eos_token_id
          )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace(prompt, "").strip()
    return response

def concept_explanation(concept):
  prompt = f"Explain the concept of {concept} in detail with example"
  return generate_response(prompt,  max_length=800)

def quiz_generator(concept):
  prompt = f"Generate 5 question about{concept} with diffrent quies type (Multiple Choice, true/false, short answer):, Give me the answer at the end:"
  return generate_response(prompt, max_length=1200)


#create gardio
with gr.Blocks() as app:
  gr.Markdown("# Educational AI Assistant")
  with gr.Tabs():
    with gr.TabItem("Concept: Explanation"):
      concept_input = gr.Textbox(label="Enter a Concept", placeholder="e.g. Machine Learning")
      explain_button = gr.Button("Explain")
      explain_output = gr.Textbox(label="Explanation", lines=10)

      explain_button.click(concept_explanation, inputs=concept_input, outputs=explain_output)

    with gr.TabItem("Quiz Generator"):
      quiz_input = gr.Textbox(label="Enter a Topic", placeholder="e.g. Machine Learning")
      quiz_button = gr.Button("Generate Quiz")
      quiz_output = gr.Textbox(label="Quiz_Questions & Answers", lines=15)

      quiz_button.click(quiz_generator, inputs=quiz_input, outputs=quiz_output)

app.launch(share=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9f5dc8a5df952313f8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
